In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For our modeling steps
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import normalize
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss
from sklearn.utils import resample
from sklearn.datasets import load_breast_cancer, load_iris, make_classification
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

from sklearn.metrics import confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report


# For demonstrative purposes
from scipy.special import logit, expit
from sklearn import datasets




In [2]:
crashes_df = pd.read_csv('Data/Traffic_Crashes_-_Crashes.csv', low_memory=False)
people_df = pd.read_csv('Data/Traffic_Crashes_-_People.csv', low_memory=False)
vehicles_df = pd.read_csv('Data/Traffic_Crashes_-_Vehicles.csv', low_memory=False)

In [3]:
crashes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 767532 entries, 0 to 767531
Data columns (total 49 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   CRASH_RECORD_ID                767532 non-null  object 
 1   RD_NO                          763205 non-null  object 
 2   CRASH_DATE_EST_I               57802 non-null   object 
 3   CRASH_DATE                     767532 non-null  object 
 4   POSTED_SPEED_LIMIT             767532 non-null  int64  
 5   TRAFFIC_CONTROL_DEVICE         767532 non-null  object 
 6   DEVICE_CONDITION               767532 non-null  object 
 7   WEATHER_CONDITION              767532 non-null  object 
 8   LIGHTING_CONDITION             767532 non-null  object 
 9   FIRST_CRASH_TYPE               767532 non-null  object 
 10  TRAFFICWAY_TYPE                767532 non-null  object 
 11  LANE_CNT                       199005 non-null  float64
 12  ALIGNMENT                     

In [ ]:
crashes_df.head()

In [ ]:
people_df.info()

In [ ]:
vehicles_df.head()

In [ ]:
unique_makes = vehicles_df['MAKE'].unique()
print(unique_makes)

In [ ]:
unique_makes = vehicles_df['MAKE'].unique()

for make in unique_makes:
    print(make)

In [ ]:
# Count the frequency of each unique vehicle make
make_counts = vehicles_df['MAKE'].value_counts()

# Sort the makes by frequency in descending order
sorted_makes = make_counts.sort_values(ascending=False)

# Select the top 25 makes
top_25_makes = sorted_makes.head(25)

# Print the top 25 most frequent makes
print(top_25_makes)

In [4]:
import pandas as pd

# Assuming you already have your DataFrame loaded as vehicles_df

# Define the top 25 most frequent makes
top_25_makes = [
    'CHEVROLET', 'UNKNOWN', 'FORD', 'NISSAN', 'HONDA',
    'TOYOTA MOTOR COMPANY, LTD.', 'TOYOTA', 'DODGE', 'HYUNDAI', 'JEEP',
    'VOLKSWAGEN', 'CHRYSLER', 'BUICK', 'LEXUS', 'KIA', 'MERCEDES-BENZ',
    'BMW', 'MAZDA', 'KIA MOTORS CORP', 'CADILLAC', 'INFINITI', 'SUBARU',
    'ACURA', 'PONTIAC', 'GENERAL MOTORS CORPORATION (GMC)'
]

# Create boolean columns for each of the top makes
for make in top_25_makes:
    vehicles_df[make] = vehicles_df['MAKE'].apply(lambda x: int(x == make))

# Use one-hot encoding to convert the boolean columns into binary encoding

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
encoded_columns = encoder.fit_transform(vehicles_df[top_25_makes])



C:\Users\Nick\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
unique_safety = people_df['SAFETY_EQUIPMENT'].unique()

for seatbelt in unique_safety:
    print(seatbelt)

In [5]:


# Define the top 25 most frequent makes
top_25_makes = [
    'CHEVROLET', 'UNKNOWN', 'FORD', 'NISSAN', 'HONDA',
    'TOYOTA MOTOR COMPANY, LTD.', 'TOYOTA', 'DODGE', 'HYUNDAI', 'JEEP',
    'VOLKSWAGEN', 'CHRYSLER', 'BUICK', 'LEXUS', 'KIA', 'MERCEDES-BENZ',
    'BMW', 'MAZDA', 'KIA MOTORS CORP', 'CADILLAC', 'INFINITI', 'SUBARU',
    'ACURA', 'PONTIAC', 'GENERAL MOTORS CORPORATION (GMC)'
]

# Create boolean columns for each of the top makes
for make in top_25_makes:
    vehicles_df[make] = vehicles_df['MAKE'].apply(lambda x: int(x == make))

# Use one-hot encoding to convert the boolean columns into binary encoding

from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse=False)
encoded_columns = encoder.fit_transform(vehicles_df[top_25_makes])

# Fill missing values in 'SAFETY_EQUIPMENT' with "UNKNOWN"
people_df['SAFETY_EQUIPMENT'].fillna("UNKNOWN", inplace=True)
# Create boolean columns for "SAFETY BELT USED" and "SAFETY BELT NOT USED"
people_df['SAFETY_BELT_USED'] = people_df['SAFETY_EQUIPMENT'].str.contains("SAFETY BELT USED", case=False)
people_df['SAFETY_BELT_NOT_USED'] = people_df['SAFETY_EQUIPMENT'].str.contains("SAFETY BELT NOT USED", case=False)

# Convert the boolean values to 1 (True) and 0 (False)
people_df['SAFETY_BELT_USED'] = people_df['SAFETY_BELT_USED'].astype(int)
people_df['SAFETY_BELT_NOT_USED'] = people_df['SAFETY_BELT_NOT_USED'].astype(int)


C:\Users\Nick\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
print(crashes_df.columns)

In [ ]:
print(vehicles_df.columns)


In [6]:
columns_to_drop = [
    'INTERSECTION_RELATED_I',
    'NOT_RIGHT_OF_WAY_I',
    'HIT_AND_RUN_I',
    'DAMAGE',
    'DATE_POLICE_NOTIFIED',
    'PRIM_CONTRIBUTORY_CAUSE',
    'SEC_CONTRIBUTORY_CAUSE',
    'STREET_NO',
    'STREET_DIRECTION',
    'STREET_NAME',
    'BEAT_OF_OCCURRENCE',
    'PHOTOS_TAKEN_I',
    'STATEMENTS_TAKEN_I',
    'DOORING_I',
    'WORK_ZONE_I',
    'WORK_ZONE_TYPE',
    'WORKERS_PRESENT_I',
    'NUM_UNITS',
    'MOST_SEVERE_INJURY',
    'LATITUDE',
    'LONGITUDE',
    'LOCATION'
]

crashes_df.drop(columns=columns_to_drop, inplace=True)


In [7]:
# Sample a portion of your data
sampled_crashes_df = crashes_df.sample(frac=0.1, random_state=42)  # Adjust the fraction as needed

# Then perform your merge operations on the sampled data
sampled_crashes_df = pd.merge(sampled_crashes_df, vehicles_df, on='CRASH_RECORD_ID', how='left')
sampled_crashes_df = pd.merge(sampled_crashes_df, people_df[['CRASH_RECORD_ID', 'SAFETY_BELT_USED', 'SAFETY_BELT_NOT_USED']],
                              on='CRASH_RECORD_ID', how='left')



In [ ]:

sample_fraction = 0.1  # 10% of the data

# Use the sample method to randomly select a fraction of your data
sampled_crashes_df = crashes_df.sample(frac=sample_fraction, random_state=42)  # Set random_state for reproducibility



In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder



# Handle missing values in sampled_crashes_df
sampled_crashes_df['INJURIES_FATAL'].fillna(0, inplace=True)  # Fill missing values in the target variable with 0
sampled_crashes_df['MAKE'].fillna('UNKNOWN', inplace=True)    # Fill missing values in 'MAKE' with 'UNKNOWN'

# Define your features and target variable
features = [
    'SAFETY_BELT_USED',
    'SAFETY_BELT_NOT_USED',
    'CHEVROLET',
    'UNKNOWN',
    'FORD',
    'NISSAN',
    'HONDA',
    'TOYOTA MOTOR COMPANY, LTD.',
    'TOYOTA',
    'DODGE',
    'HYUNDAI',
    'JEEP',
    'VOLKSWAGEN',
    'CHRYSLER',
    'BUICK',
    'LEXUS',
    'KIA',
    'MERCEDES-BENZ',
    'BMW',
    'MAZDA',
    'KIA MOTORS CORP',
    'CADILLAC',
    'INFINITI',
    'SUBARU',
    'ACURA',
    'PONTIAC',
    'GENERAL MOTORS CORPORATION (GMC)'
]

X = sampled_crashes_df[features]
y = sampled_crashes_df['INJURIES_FATAL']

# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Drop rows with missing values from both X and y
X_train.dropna(inplace=True)
y_train = y_train[X_train.index]  # Update y_train to match the modified X_train

X_test.dropna(inplace=True)
y_test = y_test[X_test.index]  # Update y_test to match the modified X_test

# Create and train a logistic regression model
logistic_regression_model = LogisticRegression(random_state=42)
logistic_regression_model.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = logistic_regression_model.predict(X_test)

# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report_str)


Accuracy: 1.00
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     70497
         1.0       0.00      0.00      0.00        84
         2.0       0.00      0.00      0.00         2

    accuracy                           1.00     70583
   macro avg       0.33      0.33      0.33     70583
weighted avg       1.00      1.00      1.00     70583



C:\Users\Nick\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nick\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nick\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# Evaluate the model's performance
accuracy = accuracy_score(y_test, y_pred)
classification_report_str = classification_report(y_test, y_pred)

# Print the results
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report_str)


In [ ]:
vehicles_df.isna().mean()

In [ ]:
people_df.INJURY_CLASSIFICATION.value_counts()

In [ ]:
def dropna_columns(df, threshold=.8):
    '''
    Drop columns from a Pandas DataFrame in which more than a certain
    percentage (default=80%) of their rows are null/NaN.
    '''
    return df.dropna(thresh=(((1-threshold) * df.shape[0])) + 1, axis=1)

In [ ]:
crashes_dropped_df = dropna_columns(crashes_df)
people_dropped_df = dropna_columns(people_df)
vehicles_dropped_df = dropna_columns(vehicles_df)

In [ ]:
crashes_dropped_df.shape

In [ ]:
people_dropped_df.shape

In [ ]:
vehicles_dropped_df.shape

In [ ]:
vehicles_dropped_df.columns

In [ ]:
people_dropped_df.columns

In [ ]:
crashes_dropped_df.columns

In [ ]:
people_dropped_df.head(12)

In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


# Identify columns with missing values and their percentages
missing_percentage = crashes_df.isnull().mean() * 100

# Set a threshold for missing values (e.g., 30%)
threshold = 30

# Columns with more than the threshold percentage of missing values
columns_to_drop = missing_percentage[missing_percentage > threshold].index

# Drop columns with high missing values
crashes_df.drop(columns=columns_to_drop, inplace=True)

# For columns with fewer missing values, you can impute them using the median or mode
columns_to_impute = ['LANE_CNT', 'INTERSECTION_RELATED_I', 'NOT_RIGHT_OF_WAY_I', 'HIT_AND_RUN_I']
for column in columns_to_impute:
    if column in crashes_df.columns:
        if crashes_df[column].dtype == 'float64':
            crashes_df[column].fillna(crashes_df[column].median(), inplace=True)
        else:
            crashes_df[column].fillna(crashes_df[column].mode()[0], inplace=True)

# Identify categorical columns
categorical_columns = crashes_df.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns with a reasonable number of categories
max_categories_for_one_hot = 10  # You can adjust this threshold
columns_for_one_hot = [col for col in categorical_columns if len(crashes_df[col].unique()) <= max_categories_for_one_hot]

# Apply one-hot encoding to selected categorical columns
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_columns = pd.DataFrame(encoder.fit_transform(crashes_df[columns_for_one_hot]))
encoded_columns.columns = encoder.get_feature_names_out(columns_for_one_hot)

# Drop the original categorical columns and concatenate the one-hot encoded columns
crashes_df.drop(columns=columns_for_one_hot, inplace=True)
crashes_df = pd.concat([crashes_df, encoded_columns], axis=1)

# Apply label encoding to remaining categorical columns
label_encoder = LabelEncoder()
for column in categorical_columns:
    if column not in columns_for_one_hot:
        crashes_df[column] = label_encoder.fit_transform(crashes_df[column])


In [ ]:
people_df.info()

In [ ]:
# Identify columns with missing values and their percentages
missing_percentage = vehicles_df.isnull().mean() * 100

# Set a threshold for missing values (e.g., 30%)
threshold = 30

# Columns with more than the threshold percentage of missing values
columns_to_drop = missing_percentage[missing_percentage > threshold].index

# Drop columns with high missing values
vehicles_df.drop(columns=columns_to_drop, inplace=True)

# For columns with fewer missing values, you can impute them using the median or mode
columns_to_impute = ['NUM_PASSENGERS', 'CMRC_VEH_I', 'LIC_PLATE_STATE', 'VEHICLE_YEAR']
for column in columns_to_impute:
    if column in vehicles_df.columns:
        if vehicles_df[column].dtype == 'float64':
            vehicles_df[column].fillna(vehicles_df[column].median(), inplace=True)
        else:
            vehicles_df[column].fillna(vehicles_df[column].mode()[0], inplace=True)

# Identify categorical columns
categorical_columns = vehicles_df.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns with a reasonable number of categories
max_categories_for_one_hot = 10  # You can adjust this threshold
columns_for_one_hot = [col for col in categorical_columns if len(vehicles_df[col].unique()) <= max_categories_for_one_hot]

# Apply one-hot encoding to selected categorical columns
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_columns = pd.DataFrame(encoder.fit_transform(vehicles_df[columns_for_one_hot]))
encoded_columns.columns = encoder.get_feature_names_out(columns_for_one_hot)  # Corrected method name

# Drop the original categorical columns and concatenate the one-hot encoded columns
vehicles_df.drop(columns=columns_for_one_hot, inplace=True)
vehicles_df = pd.concat([vehicles_df, encoded_columns], axis=1)

# Apply label encoding to remaining categorical columns
label_encoder = LabelEncoder()
for column in categorical_columns:
    if column not in columns_for_one_hot:
        vehicles_df[column] = label_encoder.fit_transform(vehicles_df[column])



In [ ]:

# Identify columns with missing values and their percentages
missing_percentage = people_df.isnull().mean() * 100

# Set a threshold for missing values (e.g., 30%)
threshold = 30

# Columns with more than the threshold percentage of missing values
columns_to_drop = missing_percentage[missing_percentage > threshold].index

# Drop columns with high missing values
people_df.drop(columns=columns_to_drop, inplace=True)

# For columns with fewer missing values, you can impute them using the median or mode
columns_to_impute = ['SEAT_NO', 'CITY', 'STATE', 'ZIPCODE', 'AGE', 'DRIVERS_LICENSE_STATE', 'DRIVERS_LICENSE_CLASS',
                     'AIRBAG_DEPLOYED', 'EJECTION', 'HOSPITAL', 'EMS_AGENCY', 'EMS_RUN_NO', 'DRIVER_ACTION',
                     'DRIVER_VISION', 'PHYSICAL_CONDITION', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY', 'PEDPEDAL_LOCATION',
                     'BAC_RESULT', 'BAC_RESULT VALUE', 'CELL_PHONE_USE']
for column in columns_to_impute:
    if column in people_df.columns:
        if people_df[column].dtype == 'float64':
            people_df[column].fillna(people_df[column].median(), inplace=True)
        else:
            people_df[column].fillna(people_df[column].mode()[0], inplace=True)

# Identify categorical columns
categorical_columns = people_df.select_dtypes(include=['object']).columns

# Apply one-hot encoding to categorical columns with a reasonable number of categories
max_categories_for_one_hot = 10  # You can adjust this threshold
columns_for_one_hot = [col for col in categorical_columns if len(people_df[col].unique()) <= max_categories_for_one_hot]

# Apply one-hot encoding to selected categorical columns
encoder = OneHotEncoder(drop='first', sparse=False)
encoded_columns = pd.DataFrame(encoder.fit_transform(people_df[columns_for_one_hot]))
encoded_columns.columns = encoder.get_feature_names_out(columns_for_one_hot)  # Corrected method name

# Drop the original categorical columns and concatenate the one-hot encoded columns
people_df.drop(columns=columns_for_one_hot, inplace=True)
people_df = pd.concat([people_df, encoded_columns], axis=1)

# Apply label encoding to remaining categorical columns
label_encoder = LabelEncoder()
for column in categorical_columns:
    if column not in columns_for_one_hot:
        people_df[column] = label_encoder.fit_transform(people_df[column])



In [ ]:
vehicles_df.head()

In [ ]:
crashes_df.head()